Goal: Investigate different combination of landmarks

Ex: glabella and chin, etc ...

In [2]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

vhr.plot.VisualizeParams.renderer = 'vscode' 

In [3]:
# -- LOAD A DATASET

dataset_name = 'mr_nirp'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

videos = constants.get_video_settings(dataset_name)
print(videos)

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

0 D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
2 D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\RGB_corrected\Subject2_motion_940.avi
3 D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\RGB_corrected\Subject2_still_940.avi
4 D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\RGB_corrected\Subject3_motion_940.avi
5 D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\RGB_corrected\Subject3_still_940.avi
6 D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\RGB_corrected\Subject4_motion_940.avi
7 D:/datasets_rppg/MR-NIRP_indoor\Subject4_still_940-004\Subject4_still_940\RGB_corrected\Subject4_still_940.avi
8 D:/datasets_rppg/MR-NIRP_indoor\Subject5_still_940-003\Subject5_still_940\RGB_corrected\Subject5_s

In [5]:
# -- PARAMETER SETTING

roi = 'mustache'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 2     # minimum number of landmarks tested
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for mr_nirp is 60 
Testing ROI = mustache with minimum 2 landmarks


# Combinations

In [9]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [54]:
all_landmarks = vhr.extraction.utils.CustomLandmarks().get_all_landmarks()
all_landmarks = pd.DataFrame(all_landmarks.items(), columns=['name','values'])
combination = all_landmarks.sample(min_len)

# iterate through the rows of the dataframe
for index, row in combination.copy().iterrows():
    print(row['name'], row['values'])
    new_row = None
    if 'left' in row['name']:
        new_row = all_landmarks[all_landmarks['name'].str.contains('right_'+'_'.join(row['name'].split('_')[1:]))]
    if 'right' in row['name']:
        new_row = all_landmarks[all_landmarks['name'].str.contains('left_'+'_'.join(row['name'].split('_')[1:]))]
    if new_row is not None:
        combination = pd.concat([combination, new_row], ignore_index=True)
combination

philtrum [2, 97, 167, 37, 0, 267, 393, 326]
lower_medial_forehead [10, 109, 108, 151, 337, 338]


,name,values
14,philtrum,"[2, 97, 167, 37, 0, 267, 393, 326]"
0,lower_medial_forehead,"[10, 109, 108, 151, 337, 338]"
